# Translating an audio from English to Spanish

In [63]:
from openai import OpenAI
from elevenlabs import generate, play, set_api_key, voices, Voice, VoiceSettings
import ffmpeg
from pydub import AudioSegment
from io import BytesIO

# Connecting to OpenAI   
client = OpenAI(api_key=API_KEY) #Replace for your OpenAI API-KEY


# Connecting to ElevenLabs   
set_api_key(API_KEY_eleven)  #Replace for your ElevenLabs API-KEY

## Translating the audio to text

In [50]:
audio_file_eng = open('arnold_speech_vision.mp3', 'rb')  # I've used one of Arnold Schwarzenegger's inspiring speeches as the input audio file

# Write an appropriate prompt to help the model understand the audio 
prompt = "the audio file is about Arnold Schwarzenegger's speech." 

In [51]:
# Create a transcription from the audio file
response = client.audio.transcriptions.create(model='whisper-1', 
                                              file = audio_file_eng, 
                                               prompt=prompt,
                                              response_format="text")


In [52]:
# Printing the full response
response

"After the Second World War, we were occupied by America, by France, by Russia, by England. And there was no food around, there was poverty, there was literally, I mean, the worst condition was worse than during the war. And so, in that kind of an environment, it made me feel as a kid always that I wanted to get out of there, and that I deserved better than that. And I always was dreaming about coming to America, America was always the land of opportunity. So I always shot for the top, no matter what it is, I was so hungry for success, I was so hungry to get out of Austria, I was so determined, I had this fire in the belly. And I always, like I said, dreamt big in bodybuilding, when I got into bodybuilding it was not just to become a bodybuilding champion, but I wanted to be the greatest bodybuilder of all times. I always shoot for the top, because I always feel like if you have a great vision, and if you think big, then you're also going to end up big. If you think little, perhaps no 

## Summarizing the main ideas of the Speech & Translating to Spanish

In [53]:
# Writing the prompt that will ask the chatbot to summarize and translate the text
translation_prompt ='Select the most inspiring sentences from the speech and translate to spanish: '

# Initiating the chatbot
text_to_spanish = client.chat.completions.create(model='gpt-3.5-turbo', 
                                                 messages =[
                                                 {'role':'user',
                                                  'content':translation_prompt + text
                                                 }]
                                                )

In [54]:
# Response retrieved
response_spanish = text_to_spanish.choices[0].message.content

response_spanish

'Trabaja como loco. No hay una píldora mágica. Me vuelve loco ir al gimnasio. Lee una hora al día. Yo iba y entrenaba cinco horas al día y también trabajaba en construcción. Tú tienes 24 horas al día. Organiza tu día. Trabaja duro. Estoy aquí para hablar sobre el éxito. Fui a la universidad. Iba y entrenaba cinco horas al día y también trabajaba en construcción porque en aquel entonces en el culturismo no había dinero. No tenía dinero para suplementos alimenticios ni nada, así que tenía que trabajar. Así que trabajé en construcción. Fui a la universidad. Entrenaba en el gimnasio y por las noches, desde las ocho de la noche hasta las doce de la noche, iba a clases de actuación cuatro veces a la semana. Así que hice todo eso. No perdí ni un solo minuto. Y es por eso que estoy aquí parado hoy. A los 20 años fui a Londres y gané el concurso de Mr. Universo como el Mr. Universo más joven de la historia. Y fue porque tenía una meta. Verás, si no tienes una visión de adónde vas y si no tienes

In [64]:
# Breaking to response into multiple lines 
response_spanish = response_spanish.replace('. ', '.\n')
print(response_spanish)

Trabaja como loco.
No hay una píldora mágica.
Me vuelve loco ir al gimnasio.
Lee una hora al día.
Yo iba y entrenaba cinco horas al día y también trabajaba en construcción.
Tú tienes 24 horas al día.
Organiza tu día.
Trabaja duro.
Estoy aquí para hablar sobre el éxito.
Fui a la universidad.
Iba y entrenaba cinco horas al día y también trabajaba en construcción porque en aquel entonces en el culturismo no había dinero.
No tenía dinero para suplementos alimenticios ni nada, así que tenía que trabajar.
Así que trabajé en construcción.
Fui a la universidad.
Entrenaba en el gimnasio y por las noches, desde las ocho de la noche hasta las doce de la noche, iba a clases de actuación cuatro veces a la semana.
Así que hice todo eso.
No perdí ni un solo minuto.
Y es por eso que estoy aquí parado hoy.
A los 20 años fui a Londres y gané el concurso de Mr.
Universo como el Mr.
Universo más joven de la historia.
Y fue porque tenía una meta.
Verás, si no tienes una visión de adónde vas y si no tienes 

## Creating a Spanish Audio from the Translated text

In [74]:
# selecting the voice id from ElevenLabs 
voice_id = VOICE_ID # #Replace for the voice id you want to use --> from elevenlabs import voices ... voices = voices(), gets you a list of all available voices

# Creating the audio file
audio = generate(response_spanish, 
                 model="eleven_multilingual_v2",  ##different languages
                 voice= Voice(
                    voice_id=voice_id ,
                     settings=VoiceSettings(
                                 stability = 0.3,similarity_boost=0.5, style = 0.75,use_speaker_boost=True)
                )
               )

## Playing the generated audio file

In [76]:
play(audio)

## Saving the file

In [77]:
def export_bytes_to_audio(bytes_data, output_path, format="mp3"):
    # Convert bytes to BytesIO object
    audio_bytes_io = BytesIO(bytes_data)

    # Load the audio from BytesIO using pydub
    audio = AudioSegment.from_file(audio_bytes_io)

    # Export the audio to the specified output path
    audio.export(output_path, format=format)

filename = 'translated_audio_file.mp3'

export_bytes_to_audio(audio, filename)